In [11]:
# Downloading MNIST Traing Data

import sys, os
sys.path.append(os.pardir) 
from DeepLearningFromScratch.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


## 손글씨 숫자 인식
### MNIST 데이터셋
- 0~9 사이의 숫자 이미지
- Train Data : 6,000장
- Test Data : 10,000장
- 이미지 데이터 : 28 x 28
- 각 픽셀 : 0~255

## 인수

1. Normalize : 픽셀값 0.0 ~ 1.0 으로 정규화
2. Flatten : 입력 이미지를 1차원으로
3. One_hot_label

In [5]:
# MNIST Image - Loading

import sys,os
sys.path.append(os.pardir)
import numpy as np
from DeepLearningFromScratch.mnist import load_mnist
from matplotlib.pyplot import imshow
from PIL import Image # PIL : Pyhton Image Library

def img_show(img) :
    pil_img = Image.fromarray(np.uint8(img)) # numpy array to PIL Data type
    pil_img.show()
    
(x_train,t_train),(x_test,t_test) = load_mnist(flatten = True, normalize = False) # Flatten : 1 Dim numpy Array

img = x_train[0]
label = t_train[0]

print(label)

print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

%matplotlib inline
img_show(img)

5
(784,)
(28, 28)


In [6]:
# MNIST Image - Reshape

img = img.reshape(28,28) # Original Shape
img.shape

(28, 28)

## 신경망의 추론 처리
- 입력층 뉴런 : 784 개 (이미지 크기 : 28 x 28 = 784)
- 출력층 뉴런 : 10개 (0 ~ 9의 숫자)
- 은닉1층 뉴런 : 50개 (임의)
- 은닉2층 뉴런 : 100개 (임의)

In [7]:
# MNIST Image - Setting Function


import pickle


# Sigmoid Function
def sigmoid(x):
    return 1/(1 + np.exp(-x))

# Softmax Function
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a/sum_exp_a
    return y


def get_data():
    (x_train,t_train),(x_test,t_test) = load_mnist(normalize = True, flatten = True, one_hot_label = False)
    return x_test, t_test


# initiate pickle file
def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network,x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

In [8]:
# MNIST Image - Accuracy

x,t = get_data()
network = init_network()

accuracy_cnt = 0                   
for i in range(len(x)):            # Probability into Numpy Array
    y = predict(network,x[i])      # Return in Numpy Array
    p = np.argmax(y)               # Result
    if p == t[i]:                  # Comparison x & t
        accuracy_cnt += 1
    
print("Accuracy : " + str(float(accuracy_cnt)/len(x)))

Accuracy : 0.9352


## 배치 처리

- 배치 : 하나로 묶은 입력 데이터 
- 이미지가 지폐의 다발 처럼 있음
- 이미지 1장당 처리 시간 줄여줌 (버스에 주는 부하 줄임)
- 큰 배열 계산 수월

### 예시
- 784  * (784 x 50) * (50 x 100) * (100 x 10)

In [11]:
# Batch Preprocessing - X shape

x,_ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']
x.shape

(10000, 784)

In [13]:
# Batch Preprocessing - X[0] shape

x[0].shape

(784,)

In [14]:
# Batch Preprocessing - W1 shape

W1.shape

(784, 50)

In [15]:
# Batch Preprocessing - W2 shape

W2.shape

(50, 100)

In [16]:
# Batch Preprocessing - W3 shape

W3.shape

(100, 10)

## 배치 처리 크기 설정

In [18]:
# Batch Preprocessing - Including Batch Size

x,t = get_data()
network = init_network()

batch_size = 100          # batch size

accuracy_cnt = 0                   
for i in range(0, len(x), batch_size):            
    x_batch = x[i:i+batch_size]
    y_batch = predict(network,x_batch)
    p = np.argmax(y_batch, axis = 1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print("Accuracy : " + str(float(accuracy_cnt)/len(x)))

Accuracy : 0.9352


In [20]:
# Batch Preprocessing - Range Reference 1
list(range(0,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [22]:
# Batch Preprocessing - Range Reference 2
list(range(0,10,3))

[0, 3, 6, 9]

In [26]:
# Batch Preprocessing - Argmax Reference 
x = np.array([[0.1,0.8,0.1],[0.3,0.1,0.6],[0.2,0.5,0.3],[0.8,0.1,0.1]])
y = np.argmax(x,axis = 1) # MAX Index (Value X)
y

array([1, 2, 1, 0], dtype=int64)